In [3]:
#엄마의 장바구니 입력
dataset = [['우유', '양파', '땅콩', '호두', '계란', '요거트'],
           ['감자', '양파', '두유', '호두', '계란', '요거트'],
           ['우유', '사과', '호두', '계란'],
           ['우유', '새우깡', '옥수수', '호두', '요거트'],
           ['옥수수', '감자', '양파', '호두', '아이스크림', '계란']]

In [5]:
!pip install mlxtend

In [11]:
#트랜잭션 데이터로 바꾼다. 
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder  # 거래를 숫자화 시킴 >> Bool(T/F 반환)

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)

In [12]:
te

TransactionEncoder()

In [13]:
te_ary

array([[False,  True, False,  True, False, False, False,  True, False,
         True,  True,  True],
       [ True,  True,  True, False, False, False, False,  True, False,
         True, False,  True],
       [False,  True, False, False,  True, False, False, False, False,
        False,  True,  True],
       [False, False, False, False, False,  True, False, False,  True,
         True,  True,  True],
       [ True,  True, False, False, False, False,  True,  True,  True,
        False, False,  True]])

In [14]:
df = pd.DataFrame(te_ary, columns=te.columns_)
df
# 5개의 transaction 

,감자,계란,두유,땅콩,사과,새우깡,아이스크림,양파,옥수수,요거트,우유,호두
0,False,True,False,True,False,False,False,True,False,True,True,True
1,True,True,True,False,False,False,False,True,False,True,False,True
2,False,True,False,False,True,False,False,False,False,False,True,True
3,False,False,False,False,False,True,False,False,True,True,True,True
4,True,True,False,False,False,False,True,True,True,False,False,True


In [7]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

#지지도가 0.6이상인 규칙을 추출한다. 
apriori(df, min_support=0.6)

,support,itemsets
0,0.8,(1)
1,0.6,(7)
2,0.6,(9)
3,0.6,(10)
4,1.0,(11)
5,0.6,"(1, 7)"
6,0.8,"(1, 11)"
7,0.6,"(11, 7)"
8,0.6,"(9, 11)"
9,0.6,"(10, 11)"


In [15]:
#지지도가 0.6이상인 규칙을 추출한다. 
apriori(df, min_support=0.6, use_colnames=True) # 해당 물품명 출력 

,support,itemsets
0,0.8,(계란)
1,0.6,(양파)
2,0.6,(요거트)
3,0.6,(우유)
4,1.0,(호두)
5,0.6,"(계란, 양파)"
6,0.8,"(계란, 호두)"
7,0.6,"(양파, 호두)"
8,0.6,"(요거트, 호두)"
9,0.6,"(우유, 호두)"


In [16]:
#아이템셋의 길이를 구하여 컬럼에 추가한다. 
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x)) # 물품별 크기(길이) 열 추가
frequent_itemsets

,support,itemsets,length
0,0.8,(계란),1
1,0.6,(양파),1
2,0.6,(요거트),1
3,0.6,(우유),1
4,1.0,(호두),1
5,0.6,"(계란, 양파)",2
6,0.8,"(계란, 호두)",2
7,0.6,"(양파, 호두)",2
8,0.6,"(요거트, 호두)",2
9,0.6,"(우유, 호두)",2


In [17]:
# 해당 물품이 2번 이상이고, 지지도가 0.8 이상인 물품 
frequent_itemsets[ (frequent_itemsets['length'] == 2) &
                   (frequent_itemsets['support'] >= 0.8) ]

,support,itemsets,length
6,0.8,"(계란, 호두)",2


In [18]:
frequent_itemsets[ frequent_itemsets['itemsets'] == {'양파', '계란'} ]

,support,itemsets,length
5,0.6,"(계란, 양파)",2


In [20]:
# 연관규칙을 생성한다. 
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
# association_rules() 함수 사용, metric(평가 지표) : lift, min_threshold : 절대값 크기가 1이상이여야 의미가 있음
rules
# 16가지 규칙 생성

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(계란),(양파),0.8,0.6,0.6,0.75,1.25,0.12,1.6
1,(양파),(계란),0.6,0.8,0.6,1.00,1.25,0.12,inf
2,(계란),(호두),0.8,1.0,0.8,1.00,1.00,0.00,inf
3,(호두),(계란),1.0,0.8,0.8,0.80,1.00,0.00,1.0
4,(양파),(호두),0.6,1.0,0.6,1.00,1.00,0.00,inf
5,(호두),(양파),1.0,0.6,0.6,0.60,1.00,0.00,1.0
6,(요거트),(호두),0.6,1.0,0.6,1.00,1.00,0.00,inf
7,(호두),(요거트),1.0,0.6,0.6,0.60,1.00,0.00,1.0
8,(우유),(호두),0.6,1.0,0.6,1.00,1.00,0.00,inf
9,(호두),(우유),1.0,0.6,0.6,0.60,1.00,0.00,1.0


In [21]:
rules[ (rules['lift'] > 1) & (rules['confidence'] >= 0.6) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(계란),(양파),0.8,0.6,0.6,0.75,1.25,0.12,1.6
1,(양파),(계란),0.6,0.8,0.6,1.00,1.25,0.12,inf
11,"(계란, 호두)",(양파),0.8,0.6,0.6,0.75,1.25,0.12,1.6
12,"(양파, 호두)",(계란),0.6,0.8,0.6,1.00,1.25,0.12,inf
13,(계란),"(양파, 호두)",0.8,0.6,0.6,0.75,1.25,0.12,1.6
14,(양파),"(계란, 호두)",0.6,0.8,0.6,1.00,1.25,0.12,inf
